In [1]:
from json import loads
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.functions import col,from_json,udf
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,MapType,FloatType,ArrayType

In [2]:
spark = SparkSession.\
        builder.\
        appName("process-data").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1024m").\
        getOrCreate()

22/11/20 16:29:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data = spark.read.parquet('hdfs://namenode:9000/tiki/Comment')

In [4]:
sample_data = data.select('value').take(1)

In [5]:
sample_data = sample_data[0].value

In [6]:
data_sample = loads(sample_data)

In [7]:
data_sample

{'id': 14930409,
 'title': 'Bình thường',
 'content': 'Self-help.                  .\nTác giả viết sách nghiên cứu về các công ty có sự tăng trưởng đột phá. \nJim Collins chỉ là người nghiên cứu viết sách. Không phải người trực tiếp làm và lãnh đạo cty. Có sự khác biệt ở đây\nNhìn lại quá khứ và nói trên biểu đồ.\nNói về đầu tư ngắn hạn thì có thể đầu tư cổ phiếu theo cách của tác giả J.C\nCòn về đầu tư lâu dài. Những cty lớn như Coca, Microsoft, Apple vẫn là sự lựa chọn tốt hơn rất nhiều.',
 'status': 'approved',
 'thank_count': 1,
 'score': 0.04241112,
 'new_score': 0.13589765,
 'customer_id': 16263933,
 'comment_count': 0,
 'rating': 3,
 'images': [{'id': 3125219,
   'full_path': 'https://salt.tikicdn.com/ts/review/bc/1c/2f/c7e9d477d1daf13c6051819a39721b1e.jpg',
   'status': 'approved'},
  {'id': 3125220,
   'full_path': 'https://salt.tikicdn.com/ts/review/68/07/d8/ee200ff88550d5aeb78bce8e75ad460d.jpg',
   'status': 'approved'}],
 'thanked': False,
 'created_at': 1644723970,
 'creat

In [8]:
schema = StructType([ 
    StructField("id",IntegerType(),True), 
    StructField("content",StringType(),True), 
    StructField("thank_count",IntegerType(),True), 
    StructField("customer_id",IntegerType(),True),
    StructField("rating",IntegerType(),True), 
    StructField("created_by",MapType(StringType(),StringType()),True),
    StructField("spid",IntegerType(),True),
    StructField("seller",MapType(StringType(),StringType()),True),
    StructField("timeline",MapType(StringType(),StringType()),True),
    StructField('product_id', IntegerType(),True),
])

In [9]:
df = data.withColumn("jsonData",from_json(col("value"),schema)) \
                   .select("jsonData.*")

In [10]:
df.createOrReplaceTempView('Comment')

In [11]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 
def cleanText(str_raw):
    # remove tags html
    str_raw = re.sub(CLEANR, ' ', str_raw)

    # remove special character
    str_raw = re.sub('\W+', ' ', str_raw)
    
    # remove number
    str_raw = re.sub("[0-9]+", "", str_raw)
    
    # remove space
    cleantext = re.sub(" +", " ", str_raw)
    return cleantext.lower()

In [12]:
spark.udf.register("cleanText", cleanText,StringType())

<function __main__.cleanText(str_raw)>

In [13]:
df_clean = spark.sql("""
    select id,content contentRaw,cleanText(content) content,thank_count,customer_id,rating,spid,
    seller.id seller_id,product_id, created_by.full_name customer_full_name,created_by.purchased_at purchased_at,
    from_unixtime(CAST(created_by.purchased_at as BIGINT), 'yyyy-MM-dd') date_purchased_at,
    timeline.review_created_date review_created_date,timeline.delivery_date delivery_date,
    datediff(timeline.review_created_date,timeline.delivery_date) review_after_delivery,
    case
        when rating >= 4 then 'Positive'
        when rating = 3 then 'Neutral'
        when rating <3 then 'Negative'
    end as sentiment,
    case
        when rating >= 4 then 2
        when rating = 3 then 1
        when rating <3 then 0
    end as lable
    from Comment c
""")

In [14]:
df_clean.write.mode('append').parquet('hdfs://namenode:9000/TikiCleaned/Comment')

In [15]:
df_comment = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Comment')

In [17]:
df_comment.count()

4243515